# Replace LEC and MWTP's CI with Statistical

In [1]:
%pwd

'D:\\15_Article\\12_PyCode'

In [2]:
%cd ..

D:\15_Article


## Import Package

In [3]:
import numpy as np
import pandas as pd

## load data

In [9]:
result_df_ndvi = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_NDVI.xlsx', index_col = 0)

In [10]:
result_df_ndvi.loc[result_df_ndvi['r2'] < 0.5, 'coeff'] = 0

In [11]:
result_df_ndvi_loc = result_df_ndvi.drop_duplicates()

In [12]:
result_df_ndvi_loc = result_df_ndvi_loc[['lat', 'lon', 'coeff']]

In [13]:
result_df_ndvi_loc.columns = ['lat', 'lon', 'coeff_ndvi']

In [15]:
result_df_ntl = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_NTL.xlsx', index_col = 0)

In [16]:
result_df_ntl.loc[result_df_ntl['r2'] < 0.5, 'coeff'] = 0

In [17]:
result_df_ntl_loc = result_df_ntl.drop_duplicates()

In [18]:
result_df_ntl_loc = result_df_ntl_loc[['lat', 'lon', 'coeff']]

In [19]:
result_df_ntl_loc.columns = ['lat', 'lon', 'coeff_ntl']

In [20]:
result_df_income = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_income_indiv.xlsx', index_col = 0)

In [21]:
result_df_income.loc[result_df_income['r2'] < 0.5, 'coeff'] = 0

In [22]:
result_df_income_loc = result_df_income.drop_duplicates()

In [23]:
result_df_income_loc = result_df_income_loc[['lat', 'lon', 'coeff']]

In [27]:
result_df_income_loc.columns = ['lat', 'lon', 'coeff_income']

In [28]:
merged_df = pd.merge(result_df_ndvi_loc, result_df_ntl_loc, on=['lat', 'lon'], how='left')
result_df_loc_merge = pd.merge(merged_df, result_df_income_loc, on=['lat', 'lon'], how='left')

In [30]:
result_df_loc_merge['MWPT_NDVI'] = result_df_loc_merge['coeff_ndvi'] / result_df_loc_merge['coeff_income'] / 120.96 * 1_000_000

In [31]:
result_df_loc_merge['MWPT_NTL'] = result_df_loc_merge['coeff_ntl'] / result_df_loc_merge['coeff_income'] / 120.96 * 1_000_000

In [32]:
result_df_loc_merge = result_df_loc_merge.groupby(['lat', 'lon']).mean()

In [33]:
result_df_loc_merge = result_df_loc_merge.reset_index()

In [34]:
result_df_loc_merge = result_df_loc_merge.replace([np.inf, -np.inf], 0)
result_df_loc_merge = result_df_loc_merge.fillna(0)

In [35]:
result_df_loc_merge.shape

(50852, 7)

In [36]:
result_df_loc_merge.describe()

,lat,lon,coeff_ndvi,coeff_ntl,coeff_income,MWPT_NDVI,MWPT_NTL
count,50852.000000,50852.000000,50852.000000,50852.000000,50852.000000,5.085200e+04,5.085200e+04
mean,35.773296,136.851359,0.000364,-0.002037,0.014759,3.790114e+02,-1.434595e+03
std,2.480900,3.331468,0.002655,0.006116,0.011206,7.491180e+04,8.196221e+04
min,24.301767,123.762947,-0.072164,-0.125678,-0.060621,-8.448932e+06,-1.175134e+07
25%,34.667910,135.207962,0.000000,-0.002577,0.008527,0.000000e+00,-1.520402e+03
50%,35.226959,136.923462,0.000000,-0.001347,0.013553,0.000000e+00,-7.223015e+02
75%,36.277805,139.635010,0.001134,0.000000,0.018700,6.111250e+02,0.000000e+00
max,45.511021,145.742340,0.054274,0.093483,0.162315,1.183274e+07,4.179183e+06


In [47]:
np.quantile(result_df_loc_merge['MWPT_NDVI'], .99)

np.float64(10033.013042983393)

In [48]:
np.quantile(result_df_loc_merge['MWPT_NTL'], .99)

np.float64(19305.65951599898)

In [49]:
np.quantile(result_df_loc_merge['MWPT_NDVI'], .01)

np.float64(-10070.724986854733)

In [50]:
np.quantile(result_df_loc_merge['MWPT_NTL'], .01)

np.float64(-23408.252851971658)

In [96]:
lines = []
for name in result_df_loc_merge.columns[2:]:
    line = [np.mean(result_df_loc_merge[name]), np.quantile(result_df_loc_merge[name], .005),
            np.quantile(result_df_loc_merge[name], .025), np.quantile(result_df_loc_merge[name], .5),
            np.quantile(result_df_loc_merge[name], .25), np.quantile(result_df_loc_merge[name], .50), 
            np.quantile(result_df_loc_merge[name], .75), np.quantile(result_df_loc_merge[name], .95),
            np.quantile(result_df_loc_merge[name], .975), np.quantile(result_df_loc_merge[name], .995),]
    lines.append(line)

In [97]:
data_summary = pd.DataFrame(lines)

In [98]:
data_summary.columns = ['Mean', "0.5% Quantile", "2.5% Quantile", "5% Quantile",
                        "25% Quantile", "50% Quantile",
                        "75% Quantile", "95% Quantile", "97.5% Quantile", "99.5% Quantile",]

In [99]:
data_summary.index = ['Local Marginal Effect Coefficient of NDVI', 'Local Marginal Effect Coefficient NTL', 'Local Marginal Effect  Coefficient Income', 'MWTP of NDVI', 'MWTP of NTL']

In [101]:
data_summary

,Mean,0.5% Quantile,2.5% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,97.5% Quantile,99.5% Quantile
Local Marginal Effect Coefficient of NDVI,0.000364,-0.009340,-0.004348,0.000000,0.000000,0.000000,0.001134,0.003586,0.005214,0.010961
Local Marginal Effect Coefficient NTL,-0.002037,-0.034131,-0.014922,-0.001347,-0.002577,-0.001347,0.000000,0.002205,0.005587,0.018962
Local Marginal Effect Coefficient Income,0.014759,-0.006948,-0.001145,0.013553,0.008527,0.013553,0.018700,0.033662,0.042993,0.066150
MWTP of NDVI,379.011353,-20557.831938,-4134.717079,0.000000,0.000000,0.000000,611.124960,2631.249265,4631.269289,18458.571230
MWTP of NTL,-1434.594657,-42463.994702,-10573.796801,-722.301526,-1520.401818,-722.301526,0.000000,1575.817577,5309.841772,40590.414596


In [102]:
data_summary.to_excel('13_Results/Table02_summaryTable.xlsx')